In [1]:
import Pkg

In [2]:
Pkg.status()

Status `~/.julia/environments/v1.10/Project.toml`
  [861a8166] Combinatorics v1.0.2
  [7073ff75] IJulia v1.24.2
  [2b0e0bc5] LanguageServer v4.5.1
  [91a5bcdd] Plots v1.40.4
  [b0d11df0] Quantikz v1.3.1
  [0525e862] QuantumClifford v0.9.4 `../../../Documents/Repos/QuantumClifford.jl`
  [295af30f] Revise v3.5.14
  [37e2e46d] LinearAlgebra


In [3]:
using QuantumClifford

In [4]:
using QuantumClifford.ECC

## QECC interface 

In [5]:
# c = Shor9()
c = Toric(2,2)

Toric(2, 2)

In [6]:
parity_checks(c)

+ X_X_XX__
+ _X_XXX__
+ X_X___XX
+ ZZ__Z_Z_
+ ZZ___Z_Z
+ __ZZZ_Z_

Copy the functions that compute logical operators here. They are not exported in `QuantumClifford.jl/src/ecc/ECC.jl`.

In [28]:
"""Logical X operations of a code."""
function logx_ops(c)
    md = MixedDestabilizer(parity_checks(c))
    logicalxview(md)
end

"""Logical Z operations of a code."""
function logz_ops(c)
    md = MixedDestabilizer(parity_checks(c))
    logicalzview(md)
end

logz_ops

In [29]:
logx_ops(c)

+ __XX____
+ _____X_X

In [30]:
logz_ops(c)

+ _Z_Z____
+ ZZ__Z__Z

In [31]:
? logicalxview

search: logicalxview



A view of the subtableau corresponding to the logical X operators. See also [`tab`](@ref), [`stabilizerview`](@ref), [`destabilizerview`](@ref), [`logicalzview`](@ref)


Note that there is a redundency in logics. We may consider provide a API that accepct logical operators.

In [32]:
parity_checks_x(c)

3×8 SparseArrays.SparseMatrixCSC{Bool, Int64} with 12 stored entries:
 1  ⋅  1  ⋅  1  1  ⋅  ⋅
 ⋅  1  ⋅  1  1  1  ⋅  ⋅
 1  ⋅  1  ⋅  ⋅  ⋅  1  1

In [33]:
parity_checks_z(c)

3×8 SparseArrays.SparseMatrixCSC{Bool, Int64} with 12 stored entries:
 1  1  ⋅  ⋅  1  ⋅  1  ⋅
 1  1  ⋅  ⋅  ⋅  1  ⋅  1
 ⋅  ⋅  1  1  1  ⋅  1  ⋅

In [34]:
parity_checks(c)

+ X_X_XX__
+ _X_XXX__
+ X_X___XX
+ ZZ__Z_Z_
+ ZZ___Z_Z
+ __ZZZ_Z_

In [35]:
parity_checks(c)

+ X_X_XX__
+ _X_XXX__
+ X_X___XX
+ ZZ__Z_Z_
+ ZZ___Z_Z
+ __ZZZ_Z_

In [36]:
stab_to_gf2(parity_checks(c))

6×16 Matrix{Bool}:
 1  0  1  0  1  1  0  0  0  0  0  0  0  0  0  0
 0  1  0  1  1  1  0  0  0  0  0  0  0  0  0  0
 1  0  1  0  0  0  1  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  1  0  0  1  0  1  0
 0  0  0  0  0  0  0  0  1  1  0  0  0  1  0  1
 0  0  0  0  0  0  0  0  0  0  1  1  1  0  1  0

## Concatenation

We consider cocantation of two small Toric codes.

In [37]:
c₁ = Toric(2,2) # inner
c₂ = Toric(2,2)# outer
k₁ = code_k(c₁)
n₁ = code_n(c₁)
n₂ = code_n(c₂)
s₁ = code_s(c₁)
s₂ = code_s(c₂)

6

In [38]:
n₁, k₁, s₁

(8, 2, 6)

Each of the $n_2$ qubits of c2 is replaced by a $n_1$-qubit c1 code.

Two kinds of checks:

1. Parity checks of c1 codes. Their number is $n_2 (n_1-k_1)$.
2. Parity checks of the c2 code, where Pauli operators are replaced by c1 codes' logical operators. Their number is $(n_2-k_2) k_1$. Note that there are $k_2$ pairs of X and Z logics.

In total, there are $n_1 n_2 - k_1 k_2$ parity checks, consistent with the code parameter of the concatenated code.

The first type of checks:

In [39]:
parity_checks(c₁)

+ X_X_XX__
+ _X_XXX__
+ X_X___XX
+ ZZ__Z_Z_
+ ZZ___Z_Z
+ __ZZZ_Z_

In [40]:
parity_checks(c₁)[1]

+ X_X_XX__

In [41]:
I ⊗ parity_checks(c₁)[1]  ⊗ I

+ _X_X_XX___

In [43]:
embed(n₁*n₂, 1:n₁, parity_checks(c₁)[1])

+ X_X_XX__________________________________________________________

In [44]:
inner_checks = Stabilizer(vcat([embed(n₁ * n₂, 1+(i-1)*n₁:i*n₁, parity_checks(c₁)[j]) for i in 1:n₂ for j in 1:s₁])) # parity checks of c₁ on each qubit of c₂

+ X_X_XX__________________________________________________________
+ _X_XXX__________________________________________________________
+ X_X___XX________________________________________________________
+ ZZ__Z_Z_________________________________________________________
+ ZZ___Z_Z________________________________________________________
+ __ZZZ_Z_________________________________________________________
+ ________X_X_XX__________________________________________________
+ _________X_XXX__________________________________________________
+ ________X_X___XX________________________________________________
+ ________ZZ__Z_Z_________________________________________________
+ ________ZZ___Z_Z________________________________________________
+ __________ZZZ_Z_________________________________________________
 ⋮
+ _________________________________________________X_XXX__________
+ ________________________________________________X_X___XX________
+ ________________________________________________ZZ__Z_Z__

The second types can be implemented by some kinds of tensor product on gf2 representation.

In [48]:
# h₂ = parity_matrix(c₂)
h₂ = stab_to_gf2(parity_checks(c₂))
phases₂ = phases(parity_checks(c₂))
h_logx₁ = stab_to_gf2(logx_ops(c₁))
phases_logx₁ = phases(logx_ops(c₁))
h_logz₁ = stab_to_gf2(logz_ops(c₁))
phases_logz₁ = phases(logz_ops(c₁))
# parity checks of c₂ with qubits repalced with logical qubits of c₁
outer_check_h = transpose(hcat([vcat(
    kron(h₂[i, 1:end÷2], h_logx₁[j, 1:end÷2]) .⊻ kron(h₂[i, end÷2+1:end], h_logz₁[j, 1:end÷2]), # X part
    kron(h₂[i, 1:end÷2], h_logx₁[j, end÷2+1:end]) .⊻ kron(h₂[i, end÷2+1:end], h_logz₁[j, end÷2+1:end]) # Z part
) for i in 1:s₂ for j in 1:k₁]...))
outer_check_phase = [UInt8(sum(h₂[i, 1:end÷2] * phases_logx₁[j]) + sum(h₂[i, end÷2+1:end] * phases_logz₁[j]) + phases₂[i]) & 0x3 for i in 1:s₂ for j in 1:k₁]
outer_checks = Stabilizer(outer_check_phase, outer_check_h)

+ __XX______________XX______________XX______XX____________________
+ _____X_X_____________X_X_____________X_X_____X_X________________
+ __________XX______________XX______XX______XX____________________
+ _____________X_X_____________X_X_____X_X_____X_X________________
+ __XX______________XX______________________________XX______XX____
+ _____X_X_____________X_X_____________________________X_X_____X_X
+ _Z_Z_____Z_Z_____________________Z_Z_____________Z_Z____________
+ ZZ__Z__ZZZ__Z__Z________________ZZ__Z__Z________ZZ__Z__Z________
+ _Z_Z_____Z_Z_____________________________Z_Z_____________Z_Z____
+ ZZ__Z__ZZZ__Z__Z________________________ZZ__Z__Z________ZZ__Z__Z
+ _________________Z_Z_____Z_Z_____Z_Z_____________Z_Z____________
+ ________________ZZ__Z__ZZZ__Z__ZZZ__Z__Z________ZZ__Z__Z________

In [49]:
checks = vcat(inner_checks, outer_checks)

+ X_X_XX__________________________________________________________
+ _X_XXX__________________________________________________________
+ X_X___XX________________________________________________________
+ ZZ__Z_Z_________________________________________________________
+ ZZ___Z_Z________________________________________________________
+ __ZZZ_Z_________________________________________________________
+ ________X_X_XX__________________________________________________
+ _________X_XXX__________________________________________________
+ ________X_X___XX________________________________________________
+ ________ZZ__Z_Z_________________________________________________
+ ________ZZ___Z_Z________________________________________________
+ __________ZZZ_Z_________________________________________________
 ⋮
+ _____X_X_____________X_X_____________X_X_____X_X________________
+ __________XX______________XX______XX______XX____________________
+ _____________X_X_____________X_X_____X_X_____X_X_________

In [50]:
checks.tab.nqubits

64